In [7]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(498, 16)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [10]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza da data

In [11]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [12]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [13]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [14]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [15]:
features.head()

,views,views_por_dia
0,28028,61.464912
1,1131,2.960733
2,1816,8.446512
3,1171,10.455357
4,1228,3.336957


In [16]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = df_limpo['date'] >= "2019-04-01"

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [18]:
title_bow_train.shape

(228, 193)

In [19]:
1 - 1277/(228*193)

0.9709799109171894

In [20]:
title_bow_train

<228x193 sparse matrix of type '<class 'numpy.float64'>'
	with 1277 stored elements in Compressed Sparse Row format>

In [ ]:
hstack - [1 2]     [3 4]   -> [1 2 3 4] - 1x4

vstack - [1 2]     [3 4]   -> [1 2]
                              [3 4] - 2x2

In [30]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [31]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((228, 195), (270, 195))

In [32]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [33]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [34]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [35]:
average_precision_score(yval, p)

0.1918043901336543

In [36]:
roc_auc_score(yval, p)

0.5848024316109421

In [ ]:
ap 0.17683995211083103, auc 0.6036474164133738 - mindf=1
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2

# 5 Active learning

70 exemplos que o modelo tenha dificuldade
30 exemplos aleatoriamente

In [38]:
df_unlabeled = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all')
df_unlabeled.shape

(674, 16)

In [39]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w


In [40]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['watch-title']

In [41]:
clean_date = df_unlabeled['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo_u['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [42]:
df_limpo_u.head()

,title,date
501,Kaggle Mercari Price Suggestion Challenge (1 p...,2018-11-02
502,OpenAI Gym and Python for Q-learning - Reinfor...,2018-10-14
503,"Dashboarding with Notebooks, Day 1: What infor...",2018-12-17
504,How To Get US- American Company H1 Visa To Get...,2019-11-23
505,Platform Overview - Machine Learning,2019-05-21


In [43]:
views = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo_u['views'] = views

In [44]:
features_u = pd.DataFrame(index=df_limpo_u.index)

In [45]:
features_u['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo_u['date']) / np.timedelta64(1, 'D')
features_u['views'] = df_limpo_u['views']
features_u['views_por_dia'] = features_u['views'] / features_u['tempo_desde_pub']
features_u = features_u.drop(['tempo_desde_pub'], axis=1)

In [46]:
features_u.head()

,views,views_por_dia
501,2167,5.472222
502,20378,49.103614
503,10435,29.729345
504,7,0.700000
505,4298,21.928571


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)


In [48]:
title_bow_u

<674x193 sparse matrix of type '<class 'numpy.float64'>'
	with 3079 stored elements in Compressed Sparse Row format>

In [49]:
Xu_wtitle = hstack([features_u, title_bow_u])

In [50]:
Xu_wtitle

<674x195 sparse matrix of type '<class 'numpy.float64'>'
	with 4409 stored elements in COOrdinate format>

In [51]:
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [52]:
df_unlabeled['p'] = pu

In [53]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w,0.093


In [54]:
mask_u = (df_unlabeled['p'] >= 0.26) & (df_unlabeled['p'] <= 1.)
mask_u.sum()

69

In [55]:
mask_u

501     False
502     False
503     False
504     False
505      True
        ...  
1179    False
1180    False
1181    False
1182    False
1183    False
Name: p, Length: 674, dtype: bool

In [56]:
df_unlabeled[mask_u]#.sort_values("p")

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.455
519,flowlightさん『TalkingData AdTracking Fraud Detec...,NaN,1.707 visualizações,Publicado em 12 de mai. de 2018,Pessoas e blogs,flowlightさん『TalkingData AdTracking Fraud Detec...,Takami Sato\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,1.707 visualizações\n\n\n\n\n\n\n\n17\n\nGosto...,https://i.ytimg.com/vi/RKXUEJVJJ-o/maxresdefau...,1280.0,720.0,資料 https://www.slideshare.net/TakanoriHayashi3...,1280.0,720.0,kaggle,/channel/UCiECS_auJLNpFsvjTi1WuxQ,0.283
527,Reinforcement Learning with TensorFlow and Uni...,NaN,545 visualizações,Publicado em 22 de nov. de 2019,Ciência e tecnologia,Reinforcement Learning with TensorFlow and Uni...,Google Developers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,545 visualizações\n\n\n\n\n\n\n\n13\n\nGostou ...,https://i.ytimg.com/vi/S-MbpQiwfls/maxresdefau...,1280.0,720.0,"Dan Goncharov, Head of 42 Robotics GDG Fremont...",1280.0,720.0,Dan Goncharov,/channel/UC_x5XG1OV2P6uZZ5FSM9Ttw,0.261
535,Identifying art through machine learning at MoMA,NaN,9.476 visualizações,Publicado em 7 de mar. de 2018,Entretenimento,#MachineLearning #Musuem #Art\n\n\n\n Ident...,Google Arts & Culture\n\n\n\n\n\n\n\n\n\n\n\n\...,9.476 visualizações\n\n\n\n\n\n\n\n49\n\nGosto...,https://i.ytimg.com/vi/SLBqVOnn9Mo/maxresdefau...,1280.0,720.0,"The Art Recognizer, built in collaboration wit...",1280.0,720.0,museum of modern art,/channel/UCGn7dlcAmH44GqycKa_3ssA,0.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,Recognizing Students at Risk of Dropping Out -...,NaN,5 visualizações,Publicado em 24 de nov. de 2019,"SME, WMG (em nome de WM Spain); LatinAutor, CM...",Recognizing Students at Risk of Dropping Out -...,Joe Spaeth\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,5 visualizações\n\n\n\n\n\n\n\n0\n\nGostou des...,https://i.ytimg.com/vi/w104zCQNB38/maxresdefau...,1280.0,720.0,"ML4VA project by Anna Spearman, Noah Collins, ...",1280.0,720.0,NaN,/channel/UCZDNp9Zik3T8DjKzarPlDpQ,0.285
1136,Real Stories from Career Switchers | CareerCon...,NaN,531 visualizações,Publicado em 15 de ago. de 2019,Ciência e tecnologia,Real Stories from Career Switchers | CareerCon...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,531 visualizações\n\n\n\n\n\n\n\n24\n\nGostou ...,https://i.ytimg.com/vi/wyU9GTKSO0g/maxresdefau...,1280.0,720.0,"""Listen to a panel of speakers who successfull...",1280.0,720.0,careercon,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.447
1138,"NIPS 2017 Test of Time Award ""Machine learning...",NaN,13.474 visualizações,Publicado em 7 de mar. de 2018,Educação,"NIPS 2017 Test of Time Award ""Machine learning...",Preserve Knowledge\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,13.474 visualizações\n\n\n\n\n\n\n\n138\n\nGos...,https://i.ytimg.com/vi/x7psGHgatGM/maxresdefau...,1280.0,720.0,,1280.0,720.0,NIPS 2017,/channel/UC9p_wQs8b8SHvfJSuuxEnvw,0.372
1168,Learn Deep Learning Online from IBM,NaN,3.641 visualizações,Public

In [57]:
dificeis = df_unlabeled[mask_u]

In [58]:
aleatorios = df_unlabeled[~mask_u].sample(31, random_state=0)

In [59]:
pd.concat([dificeis, aleatorios]).to_csv("active_label1.csv")

In [60]:
dificeis.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.455
519,flowlightさん『TalkingData AdTracking Fraud Detec...,NaN,1.707 visualizações,Publicado em 12 de mai. de 2018,Pessoas e blogs,flowlightさん『TalkingData AdTracking Fraud Detec...,Takami Sato\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,1.707 visualizações\n\n\n\n\n\n\n\n17\n\nGosto...,https://i.ytimg.com/vi/RKXUEJVJJ-o/maxresdefau...,1280.0,720.0,資料 https://www.slideshare.net/TakanoriHayashi3...,1280.0,720.0,kaggle,/channel/UCiECS_auJLNpFsvjTi1WuxQ,0.283
527,Reinforcement Learning with TensorFlow and Uni...,NaN,545 visualizações,Publicado em 22 de nov. de 2019,Ciência e tecnologia,Reinforcement Learning with TensorFlow and Uni...,Google Developers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,545 visualizações\n\n\n\n\n\n\n\n13\n\nGostou ...,https://i.ytimg.com/vi/S-MbpQiwfls/maxresdefau...,1280.0,720.0,"Dan Goncharov, Head of 42 Robotics GDG Fremont...",1280.0,720.0,Dan Goncharov,/channel/UC_x5XG1OV2P6uZZ5FSM9Ttw,0.261
535,Identifying art through machine learning at MoMA,NaN,9.476 visualizações,Publicado em 7 de mar. de 2018,Entretenimento,#MachineLearning #Musuem #Art\n\n\n\n Ident...,Google Arts & Culture\n\n\n\n\n\n\n\n\n\n\n\n\...,9.476 visualizações\n\n\n\n\n\n\n\n49\n\nGosto...,https://i.ytimg.com/vi/SLBqVOnn9Mo/maxresdefau...,1280.0,720.0,"The Art Recognizer, built in collaboration wit...",1280.0,720.0,museum of modern art,/channel/UCGn7dlcAmH44GqycKa_3ssA,0.510


In [61]:
aleatorios.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
1148,Machine Learning Analysis of Virginia Traffic ...,NaN,9 visualizações,Publicado em 23 de nov. de 2019,Pessoas e blogs,Machine Learning Analysis of Virginia Traffic ...,Eric Yu\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando....,9 visualizações\n\n\n\n\n\n\n\n0\n\nGostou des...,https://i.ytimg.com/vi/xp2toRDr_Es/maxresdefau...,1280.0,720.0,UVA undergraduate machine learning project tha...,1280.0,720.0,traffic accidents,/channel/UCuXxRifoHCJ-R7VJfvqPnwQ,0.051
559,What's Kaggle?,NaN,38.410 visualizações,Publicado em 13 de jun. de 2019,Ciência e tecnologia,What's Kaggle?,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,38.410 visualizações\n\n\n\n\n\n\n\n126\n\nGos...,https://i.ytimg.com/vi/TNzDMOg_zsw/hqdefault.jpg,480.0,360.0,Stop staring at a blank cell. Kaggle makes it ...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.052
1097,Kaggle Predictive Modeling Competition,NaN,75 visualizações,Publicado em 10 de abr. de 2019,Pessoas e blogs,Kaggle Predictive Modeling Competition,Vishal Lala\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,75 visualizações\n\n\n\n\n\n\n\n2\n\nGostou de...,https://i.ytimg.com/vi/umobFbd8wfA/maxresdefau...,1280.0,720.0,Visit https://www.kaggle.com/c/lalapaceairbnb ...,1280.0,720.0,Kaggle,/channel/UCuphBjOdP7s_PMfupmGruxw,0.133
889,Kaggle Live Coding: Automating report generati...,NaN,1.725 visualizações,Transmitido ao vivo em 11 de out. de 2019,Ciência e tecnologia,Kaggle Live Coding: Automating report generati...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.725 visualizações\n\n\n\n\n\n\n\n66\n\nGosto...,https://i.ytimg.com/vi/kDzEZihQFig/hqdefault.jpg,480.0,360.0,This week Rachael will continue to work on her...,1280.0,720.0,reading group,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.066
562,Data Science requires Patience,NaN,21 visualizações,Publicado em 24 de nov. de 2019,Pessoas e blogs,#DataScience #MachineLearning\n\n\n\n Data ...,Przemek Chojecki\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,21 visualizações\n\n\n\n\n\n\n\n2\n\nGostou de...,https://i.ytimg.com/vi/TU9I7lC5mXA/maxresdefau...,1280.0,720.0,"Nothing comes for free, especially in technica...",1280.0,720.0,data science job,/channel/UCEq0oQsS-voRnSWbcviIDGA,0.119
